In [13]:
import numpy as np
import pandas as pd

In [14]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder

In [15]:
df=pd.read_csv('covid_toy.csv')

In [16]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


#### age is a numerical column with no missing values so we dont apply transformation on it
#### fever is a numerical column with missing values so we apply SimpleImputer
#### gender and city are nominal categorical columns so we apply OneHotEncoding 
#### cough is ordinal categorical column so we apply ordinal encoding
#### has_covid is categorical target variable so we apply label encoder

# Train Test Split

In [17]:
from sklearn.model_selection import train_test_split
X=df.drop(['has_covid'], axis=1)
y=df['has_covid']

In [18]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2,random_state=0)

# Without using column transformer

In [19]:
# simple imputer to fever col
si=SimpleImputer()
X_train_fever=si.fit_transform(X_train[['fever']])

#test data
X_test_fever=si.transform(X_test[['fever']])

X_train_fever.shape, X_test_fever.shape

((80, 1), (20, 1))

In [20]:
# ordinal encoding on cough
oe=OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough=oe.fit_transform(X_train[['cough']])

#test data
X_test_cough=oe.transform(X_test[['cough']])

X_train_cough.shape, X_test_cough.shape

((80, 1), (20, 1))

In [21]:
# onhehotencoding on gender and city
ohe=OneHotEncoder(drop='first',sparse_output=False)
X_train_gender_city=ohe.fit_transform(X_train[['gender','city']])

#test data
X_test_gender_city=ohe.transform(X_test[['gender','city']])

X_train_gender_city.shape, X_test_gender_city.shape

((80, 4), (20, 4))

In [24]:
# label encoder on has_covid
le=LabelEncoder()
y_train_covid=le.fit_transform(y_train)

#test data
y_test_covid=le.transform(y_test)

y_train_covid.shape, y_test_covid.shape

((80,), (20,))

In [25]:
#Extracting age column
X_train_age=X_train.drop(columns=['gender','fever','cough','city'])

#test data
X_test_age=X_test.drop(columns=['gender','fever','cough','city'])

X_train_age.shape, X_test_age.shape

((80, 1), (20, 1))

In [38]:
# concatenating all the numpy arrays
X_train_transformed=np.concatenate((X_train_age,X_train_cough,X_train_fever,X_train_gender_city),axis=1)

#test data
X_test_transformed=np.concatenate((X_test_age,X_test_cough,X_test_fever,X_test_gender_city), axis=1)

In [39]:
X_train_transformed

array([[ 22.        ,   0.        ,  99.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [ 56.        ,   1.        , 104.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [ 31.        ,   0.        ,  98.        ,   0.        ,
          0.        ,   1.        ,   0.        ],
       [ 75.        ,   1.        , 104.        ,   0.        ,
          1.        ,   0.        ,   0.        ],
       [ 72.        ,   0.        ,  99.        ,   1.        ,
          0.        ,   0.        ,   0.        ],
       [ 66.        ,   1.        ,  99.        ,   1.        ,
          0.        ,   0.        ,   0.        ],
       [ 14.        ,   1.        , 101.        ,   1.        ,
          0.        ,   0.        ,   0.        ],
       [ 10.        ,   1.        ,  98.        ,   0.        ,
          0.        ,   1.        ,   0.        ],
       [ 24.        ,   0.        ,  98.        ,   1.        ,
          0.    

In [40]:
X_test_transformed

array([[ 19.        ,   0.        , 100.        ,   0.        ,
          0.        ,   1.        ,   0.        ],
       [ 25.        ,   0.        , 104.        ,   1.        ,
          0.        ,   0.        ,   0.        ],
       [ 42.        ,   0.        , 101.        ,   1.        ,
          1.        ,   0.        ,   0.        ],
       [ 81.        ,   0.        , 101.        ,   0.        ,
          0.        ,   0.        ,   1.        ],
       [  5.        ,   0.        , 102.        ,   1.        ,
          0.        ,   1.        ,   0.        ],
       [ 27.        ,   0.        , 100.        ,   1.        ,
          0.        ,   1.        ,   0.        ],
       [ 69.        ,   0.        , 103.        ,   0.        ,
          0.        ,   1.        ,   0.        ],
       [ 34.        ,   1.        ,  98.        ,   1.        ,
          0.        ,   1.        ,   0.        ],
       [ 60.        ,   0.        ,  99.        ,   0.        ,
          0.    

# Using Column Transformer

In [41]:
from sklearn.compose import ColumnTransformer

In [43]:
transformer=ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(),['fever']),
    ('tnf2', OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3', OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [46]:
X_train_trans=transformer.fit_transform(X_train)
X_test_trans=transformer.transform(X_test)

In [47]:
X_train_trans.shape

(80, 7)

In [48]:
X_test_trans.shape

(20, 7)

##### transformer=ColumnTransformer(transformers=[
#####    ('transformer_name', Transformation_applied, ['column_name'])
##### ]remainder='passthrough')
##### remainder is for the remaining columns
##### either they can passthrough or they can be dropped
##### passthrough will let them remain as they are the transformations won't be applied on them